In [ ]:
!pip install transformers torch

In [ ]:
import pandas as pd
import json
import csv

# Данные

In [ ]:
data_list = []
com = []

csvfile = open('/content/sample_data/dataset.csv', "r")
f = csv.reader(csvfile, delimiter='|')
p = {}
for i in f:
  if i[1].isdigit():
    data = {
                  'Rating': i[1],
                  'Text': i[0],
                 }
    if data not in data_list:
      data_list.append(data)
      com.append(len(i[0]))

df = pd.DataFrame(sorted(data_list[1:], key=lambda x: int(x["Rating"])))

df

,Rating,Text
0,1,Ничего из заявленного не соответствует тому ч...
1,1,"Хороший отель, останавливаюсь , вернее остана..."
2,1,Очень не приятно когда у сотрудников отеля на...
3,1,"Диван был грязный, сантехника толком не работ..."
4,1,"необоснованные штрафы с гостей, распространен..."
...,...,...
37762,10,"Отель со свежим ремонтом. Все чисто, красиво...."
37763,10,"персонал, место, завтраки, чистота ничего"
37764,10,Останавливаюсь уже не первый год. Отличный от...
37765,10,"персонал, район, сервис"


# Подготовка текста

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Загрузка стоп-слов и инициализация лемматизатора
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

# Функция для предобработки текста
def preprocess_text(text):
    # Приведение текста к нижнему регистру
    text = text.lower()

    # Удаление ссылок и спецсимволов
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#','', text)

    # Удаление чисел и пунктуации
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    # Токенизация текста
    words = word_tokenize(text)

    # Удаление стоп-слов и лемматизация
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return " ".join(words)

# Пример использования
df['cleaned_review'] = df['Text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Инициализация TF-IDF векторайзера
tfidf = TfidfVectorizer(max_features=5000)

# Преобразование обучающих данных
X = tfidf.fit_transform(df['cleaned_review'])
df

,Rating,Text,cleaned_review
0,1,Ничего из заявленного не соответствует тому ч...,заявленного соответствует тому реальности номе...
1,1,"Хороший отель, останавливаюсь , вернее остана...",хороший отель останавливаюсь вернее останавлив...
2,1,Очень не приятно когда у сотрудников отеля на...,очень приятно сотрудников отеля уму финансовая...
3,1,"Диван был грязный, сантехника толком не работ...",диван грязный сантехника толком работала мини ...
4,1,"необоснованные штрафы с гостей, распространен...",необоснованные штрафы гостей распространение ф...
...,...,...,...
37762,10,"Отель со свежим ремонтом. Все чисто, красиво....",отель свежим ремонтом чисто красиво просторный...
37763,10,"персонал, место, завтраки, чистота ничего",персонал место завтраки чистота
37764,10,Останавливаюсь уже не первый год. Отличный от...,останавливаюсь первый год отличный отель отлич...
37765,10,"персонал, район, сервис",персонал район сервис


# Обучение модели

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
reviews = df['cleaned_review'].to_list()
ratings = df['Rating'].to_list()

# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.2, random_state=32)

# Преобразование текста в TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Модель классификации
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Предсказания и оценка
y_pred = model.predict(X_test_tfidf)
#print(y_pred)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


test = ['мне понравилось', "Отель ужас", 'Фен не работал. В остальном, всё неплохо', 'Дверь сломалась. Ресепшен ничем не мог помочь. Отдых испорчен.', "Так красиво! Уборка каждый день, так что сюда приятно возращаться.", "Не выдавали карту. Прачечная не работала. Отдых здесь мне не понравился."]
test_tfidf = vectorizer.transform(test)
pred = model.predict(test_tfidf)
print(pred)

Accuracy: 0.4551231135822081
              precision    recall  f1-score   support

           1       0.43      0.26      0.32       232
          10       0.59      0.88      0.70      3242
           2       0.00      0.00      0.00        72
           3       0.25      0.03      0.05       182
           4       0.11      0.03      0.04       229
           5       0.09      0.03      0.05       335
           6       0.11      0.03      0.05       335
           7       0.18      0.17      0.17       715
           8       0.24      0.23      0.23      1070
           9       0.23      0.12      0.15      1142

    accuracy                           0.46      7554
   macro avg       0.22      0.18      0.18      7554
weighted avg       0.37      0.46      0.39      7554

['10' '1' '9' '10' '10' '10']


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
